In [ ]:
import glob
import os
from osgeo import ogr
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import os
import rasterio
import cv2
import geopandas as gpd

In [ ]:
'''
The below code does the following:
     - Get the names of the predictive feature rasters
     - Get the names of the shapefile polygons which represent the images
     - iterate over the predictive feature rasters
     - create a directory with the name of the predictive feature raster
     - clip each predictive feature raster with the polygons and save it in created directories from the previous step
 
 
 Input:
 
you need to enter the following:
     - directory of the splitted polygons shp_path.
     - directory of the predictive feature rasters
     - directiry to save the cutted images
     
     
 Ouptut:
     - Cutted images saved in directories with the same name of the predictive features

'''

# shapefile path
shp_path=r"D:\USER\seleem\NRW\Koeln\Python\Data_preperation\Divided_polygons\IMG_256\Training"
os.chdir(shp_path)
shp_file = glob.glob('*.shp')

# predictive feature path
predictive_rasters_path=r"D:\USER\seleem\NRW\Koeln\Python\Data_preperation\Normalized"
os.chdir(predictive_rasters_path)
predictive_rasters = glob.glob('*.tif')

#directory to save the images
cutted_images_path=r"D:\USER\seleem\NRW\Koeln\Python\Data_preperation\Cutting_raster\Github\Training"

for raster in predictive_rasters:
    print(str(raster))
    os.chdir(predictive_rasters_path)
    ds = gdal.Open(raster) # open a raster with several bands, each band  represent one predictive feature
    gt= ds.GetGeoTransform() #get the transformation data
    proj = ds.GetProjection() #get the projection

    band = ds.GetRasterBand(1) #read the first band 
    array = band.ReadAsArray() #read the first band as an array
    
    #plt.figure()  #plot the raster to check that you every thing is working well
    #plt.imshow(array)
    
    #create folders to save the images 
    os.mkdir(os.path.join(cutted_images_path,str(raster[11:-4])))
     
    
    
    # clip
    
    for shp in shp_file:
        #print(str(file))
        os.chdir(shp_path)
        ds2 = ogr.Open(shp, 1)
        layer = ds2.GetLayer()
        shp_ds=gpd.read_file(shp)
        #print(shp_ds['Label'][0])
        #index+=1
        # we will clip the raster with each polygon and save the flooded and notflooded locations in different folders
        # we will check the label, if label =0 then this is not flooded location
        #if shp_ds['Label'][0] == 0 : # clip and save not flooded locations
             #Save the feature to a new shapefile
        dsClip = gdal.Warp(os.path.join(os.path.join(cutted_images_path,str(raster[11:-4])),str(shp[:-4])+".tif")
                           , ds, cutlineDSName = shp,
                       cropToCutline = True, dstNodata = np.nan)      
    

## Rainfall and Water depth data

In [ ]:
# to generate random number list
import random
 
#generate a random rainfall list (rainfall depth ranges from 20 to 150 mm) with the same number of the images
rainfall = [random.randrange(20, 160, 10) for i in range(len(shp_file))]
#rainfall = [random.choice[20,30,40,60,70,80,90,110,120,130,150] for i in range(len(shp_file))]
# Replace rainfall depths (50,100, and 140) with other depths. As these are going to be used to test the model
for i,n in enumerate(rainfall):
    if n==50:
        rainfall[i]=random.randrange(20, 50, 10)
    if n==100:
        rainfall[i]=random.randrange(60, 100, 10)
    if n==140:
        rainfall[i]=random.randrange(110, 140, 10)
# printing result
print ("Random number list is : " +  str(rainfall))

In [ ]:
# save the rainfall list as a pickle
import pickle

with open('Rainfall.pkl', 'wb') as f:
    pickle.dump(rainfall, f)

In [ ]:
import os

count=0
for file in shp_file:
    print(str(file)) 
    ds2 = ogr.Open(file, 1)
    layer = ds2.GetLayer()
    
    # path to water depth files
    path=r'D:\user\seleem\BWB\TELEMAC\Hotspot0\run\2m'
    path=os.path.join(path, str(rainfall[count]) + "mm","temp_result", "water_depth.tif")

    #print(path)
    ds = gdal.Open(path)
    gt= ds.GetGeoTransform()
    proj = ds.GetProjection()

    band = ds.GetRasterBand(1)
    array = band.ReadAsArray()
    #array=np.where(array<0, np.nan,array)
    #plt.figure()
    #plt.imshow(array)
    file_name="WD_"+str(file[:-4])+"_"+str(rainfall[count])+"mm.tif"
    #print(file_name)
    
    output_path=os.join.path(cutted_images_path,"Water_depth",file_name)
    #print("Output path:",output_path)
       
    dsClip = gdal.Warp(output_path, ds, cutlineDSName = file,
                   cropToCutline = True, dstNodata = np.nan)
    count+=1
    